In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import tensorflow as tf
import os
import editdistance
import re

In [ ]:
path = "drive/MyDrive/Lipread"
pred_path = os.path.join(path, 'pred.npy')
test_path = os.path.join(path, 'y_test.npy')

In [ ]:
preds = np.load(pred_path)
tests = np.load(test_path)

In [ ]:
vocab = [x for x in "abcdefghijklmnopqrstuvwxyz "]
char_to_num = tf.keras.layers.StringLookup(vocabulary=vocab, oov_token="")
num_to_char = tf.keras.layers.StringLookup(vocabulary=vocab, oov_token="", invert=True)

In [ ]:
def decode_ctc_predictions(predictions, blank_token):
    decoded_output = []
    previous_char = None

    for char in predictions:
        if char == blank_token:
            continue
        if char != previous_char:
            decoded_output.append(char)
        previous_char = char
    return ''.join(decoded_output)

In [ ]:
text_pred = num_to_char(preds[0])
text_true = num_to_char(tests[0])
riel_pred = np.array(text_pred)
riel_pred = [i.decode('utf-8') for i in riel_pred]
riel_true = np.array(text_true)
riel_true = [i.decode('utf-8') for i in riel_true]
decoded_result = decode_ctc_predictions(riel_pred, blank_token='')
decoded_true = decode_ctc_predictions(riel_true, blank_token='')
print("Non decoded result:", tf.strings.reduce_join(text_pred))
print("Decoded result:", decoded_result)
print("True result:", decoded_true)

Non decoded result: tf.Tensor(b'place rrreed   at    q     fffoouurrr     now', shape=(), dtype=string)
Decoded result: place red at q four now
True result: place red in v four now


In [ ]:
text_preds = []
text_trues = []
for i in range(len(preds)):
    text_pred = num_to_char(preds[i])
    text_true = num_to_char(tests[i])
    riel_pred = np.array(text_pred)
    riel_pred = [i.decode('utf-8') for i in riel_pred]
    riel_true = np.array(text_true)
    riel_true = [i.decode('utf-8') for i in riel_true]
    decoded_result = decode_ctc_predictions(riel_pred, blank_token='')
    decoded_true = decode_ctc_predictions(riel_true, blank_token='')
    text_preds.append(decoded_result)
    text_trues.append(decoded_true)

In [ ]:
def normalize_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def wer(reference, hypothesis):
    reference = normalize_text(reference)
    hypothesis = normalize_text(hypothesis)

    ref_words = reference.split()
    hyp_words = hypothesis.split()

    edit_distance = editdistance.eval(ref_words, hyp_words)

    wer_value = edit_distance / len(ref_words)
    return wer_value

def average_wer(references, hypotheses):
    total_wer = 0
    for ref, hyp in zip(references, hypotheses):
        total_wer += wer(ref, hyp)

    average_wer_value = total_wer / len(references)
    return average_wer_value

In [ ]:
def normalize_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def cer(reference, hypothesis):
    reference = normalize_text(reference)
    hypothesis = normalize_text(hypothesis)

    ref_chars = list(reference.replace(' ', ''))
    hyp_chars = list(hypothesis.replace(' ', ''))

    edit_distance = editdistance.eval(ref_chars, hyp_chars)

    cer_value = edit_distance / len(ref_chars)
    return cer_value

def average_cer(references, hypotheses):
    total_cer = 0
    for ref, hyp in zip(references, hypotheses):
        total_cer += cer(ref, hyp)

    average_cer_value = total_cer / len(references)
    return average_cer_value


In [ ]:
average_wer_value = average_wer(text_trues, text_preds)
print(f"Average WER: {average_wer_value:.3f}")
average_cer_value = average_cer(text_trues, text_preds)
print(f"Average CER: {average_cer_value:.3f}")

Average WER: 0.224
Average CER: 0.101
